In [1]:
from datetime import datetime, timedelta
import pandas as pd



click_stream = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/click_stream_final.csv')
transaction = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/transaction_final.csv')
product = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/product_final.csv')
customer = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/customer_final.csv')

tem = pd.merge(click_stream[['session_id', 'event_time', 'event_name']], transaction[['session_id', 'customer_id']].drop_duplicates(), on = 'session_id', how = "left")
tem2 = pd.merge(tem[tem.customer_id.notnull()], customer[['customer_id', 'first_join_date']], on = 'customer_id', how= 'left')




In [2]:
# 시간 복잡도 개선 이전 함수
# def bounce_judge(df):
#     #측정 기간 설정
#     start_date = pd.to_datetime('2020-07-01')
#     end_date = pd.to_datetime('2022-06-30')
    
#     #측정 기간에 해당하는 월 수: 24개
#     num_of_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1
    
#     #이탈 판단 기준(14일 = 60 * 60 * 24 * 14 = 1209600 초)
#     criterion = 1209600

#     # 월별 컬럼 이름 생성
#     column_names = [(start_date + pd.DateOffset(months=i)).strftime('%Y-%m') for i in range(num_of_months)]

#     #데이터의 시간 데이터 타입 초기화
#     df.event_time = pd.to_datetime(df.event_time)

#     # 빈 데이터프레임 생성
#     res = pd.DataFrame(columns=column_names)

#     #customer_id를 기준으로 반복
#     for customer in df.customer_id.unique():
        
#         #베이스 딕셔너리 생성
#         tem = {month : 0 for month in sorted(column_names)}

#         #조건에 맞는 데이터로그 불러오기
#         log = df[(df.customer_id == customer) & (df.event_name == 'HOMEPAGE') & (pd.to_datetime(df.event_time.dt.date) <= end_date) & (pd.to_datetime(df.event_time.dt.date) >= start_date)].sort_values('event_time').reset_index(drop = True)

#         # 정해진 기간에 데이터가 없을 경우 log.loc[0, 'first_join_date'] 를 호출 하는 과정에서 오류 발생 함. 이를 방지하고자 데이터의 개수가 0개이면 넘어감
#         if len(log) == 0:
#             pass

#         else:    # 2020-07월 이전에 계정을 생성한 경우
#             if pd.to_datetime(log.loc[0, 'first_join_date']) < start_date:

#                 #방문 기록이 없는 달 추출하여 집합으로 저장
#                 date_set = set([date.strftime('%Y-%m') for date in log.event_time])
#                 bounce_month = set(column_names) - date_set
        
#                 #homepage 재방문 시간 계산
#                 log['diff'] = log.event_time.diff().dt.total_seconds()

#                 #첫번째 인덱스에는 start_date 부터의 시간 간격 측정
#                 log.iloc[0,4] = (pd.Timestamp(log.iloc[0,3], tz= 'UTC') - pd.Timestamp('2020-07-01 0:0:0', tz= 'UTC')).total_seconds()

#                 #bounce_month 집합에 마지막 접속일 부터 10일 지난 시점의 달 추가(합집합)
#                 bounce_month = bounce_month.union(set([date.strftime('%Y-%m') for date in log.loc[(log[log['diff'] > criterion].index - 1), 'event_time'] + timedelta(days=14)]))

#             # 2020-07월 이후에 계정을 생성한 경우
#             else:
                
#                 #start_date 이후 first_join_date 이전 월 추출(이탈 아닌것으로 확인해야하기 때문에)
#                 date_range = pd.date_range(start=start_date, end=log.first_join_date.iloc[0], freq='M')
#                 extracted_months = set(date.strftime('%Y-%m') for date in date_range)

#                 #방문 기록이 없는 달 추출하여 집합으로 저장
#                 date_set = set([date.strftime('%Y-%m') for date in log.event_time])
#                 bounce_month = set(column_names) - date_set - extracted_months
                    
#                 #homepage 재방문 시간 계산
#                 log['diff'] = log.event_time.diff().dt.total_seconds()

#                 #첫번째 인덱스에는 first_join_date 부터의 시간 간격 측정
#                 log.iloc[0,4] = (pd.Timestamp(log.iloc[0,3], tz= 'UTC') - pd.Timestamp(log.loc[0, 'first_join_date'], tz= 'UTC')).total_seconds()

#                 #bounce_month 집합에 마지막 접속일 부터 10일 지난 시점의 달 추가(합집합)
#                 bounce_month = bounce_month.union(set([date.strftime('%Y-%m') for date in log.loc[(log[log['diff'] > criterion].index - 1), 'event_time'] + timedelta(days=14)]))

#         #이탈(1) 입력
#         for month in bounce_month:
#             tem[month] = 1
    
#         #customer_id 와 key에 대해 이탈여부 매칭하는 데이터프레임 만들기
#         for key in tem:
#             res.loc[customer, key] = int(tem[key])
    
#     return res

In [285]:
#함수 사용에 필요한 특성
#click_stream
#   event_name: 'HOMEPAGE' 인 이벤트만 조회하기 때문에
#   event_time: 재방문에 걸린 시간 조회하기 위해
#   session_id: transaction 테이블과 병합하여 customer_id를 얻기 위해
#transaction
#   session_id: click_stream 테이블과 병합하여 customer_id를 얻기 위해
#   customer_id: customer 테이블과 병합하여 first_join_date 를 얻기 위해
#customer
#   customer_id: click_stream 테이블과 병합하여 first_join_date 를 얻기 위해
#   first_join_date: 가입 시기를 알기 위해



def bounce_judge(df, days_cri = 10):
    # 측정 기간 설정
    start_date = pd.to_datetime('2020-08-01')
    end_date = pd.to_datetime('2022-07-31')
    
    # 측정 기간에 해당하는 월 수: 24개
    num_of_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1
    
    # 이탈 판단 기준(days_cir = 기준이 되는 일)
    criterion = 60 * 60 * 24 * days_cri

    # 월별 이름 생성
    column_names = [(start_date + pd.DateOffset(months=i)).strftime('%Y-%m') for i in range(num_of_months)]

    # 데이터의 시간 데이터 타입 초기화
    df.event_time = pd.to_datetime(df.event_time)

    # 빈 딕셔너리 생성
    res = {}

    # customer_id를 기준으로 그룹화
    grouped = df[(df.event_name == 'HOMEPAGE') & (pd.to_datetime(df.event_time.dt.date) <= end_date) & (pd.to_datetime(df.event_time.dt.date) >= start_date)].groupby('customer_id')

    #grouped 에 포함되지 않는 데이터 처리
    excluded_ids_set = set(df[~df.customer_id.isin(grouped.groups.keys())]['customer_id']) #grouped 에 포함되지 않는 customer_id 집합
    first_join_dates = df[df.customer_id.isin(excluded_ids_set)].groupby('customer_id').first_join_date.first() #위 집합에 포함된 customer_id의 first_join_dates
    
    
    # excluded_ids를 순회하면서 이탈 여부 결정
    for customer in excluded_ids_set:
        f_j = pd.to_datetime(first_join_dates[customer])
        
        #start_date 이전에 가입했지만 2020/07~2022/06 기간동안 접속 안한 경우 이탈(1)
        if f_j < start_date:
            res[customer] = [1] * num_of_months

        #start_date 이후에 가입하여 2020/07~2022/06 기간동안 데이터가 없는 경우 이탈X(0)            
        elif f_j > end_date:
            res[customer] = [0] * num_of_months


    
    # 각 그룹 별로 작업 수행
    for customer, log in grouped:
        tem = {month: 0 for month in sorted(column_names)}
            
        # 방문 기록이 없는 달 추출하여 집합으로 저장
        date_set = set([date.strftime('%Y-%m') for date in log.event_time])
        bounce_month = set(column_names) - date_set

        # homepage 재방문 시간 계산
        log['diff'] = log.event_time.diff().dt.total_seconds()
        log.reset_index(drop = True, inplace = True)

            
        first_index = log.index[0]
        log_index = log[log['diff'] > criterion].index
            
        # .diff() 함수 적용시 첫 행에 나오는 nan 값에 대한 처리
        # fisrt_join date < start_date 일 경우 start_date ~ 첫번째 homepage event 발생 시점까지의 시간을 첫 행에 입력
        if pd.to_datetime(log.loc[first_index, 'first_join_date']) < start_date:
            log.at[first_index, 'diff'] = (log.at[first_index, 'event_time'] - pd.Timestamp('2020-08-01 0:0:0', tz='UTC')).total_seconds()

            #위에서 계산한 값이 criterion 값보다 클 경우 아래 로직에서 (0 - 1)이 계산 되어 음수 인덱스로 인한 에러 발생 / 첫 행에 대해서는 기준 보다 클 때 첫번째 event_time 의 연/월에 이탈(1) 처리
            if log.at[first_index, 'diff'] > criterion:
                bounce_month.add(log.at[first_index, 'event_time'].strftime('%Y-%m'))
                log_index = log[log['diff'] > criterion].index.drop(0) #아래 로직에서 오류 발생하지 않게 0번 인덱스 드랍(위에서 해당 행에 대해 이탈 여부를 처리함)

        # fisrt_join date >= start_date 일 경우 start_date ~ fisrt_join_date 시점까지의 연/월에 대해 이탈X(0) 처리 / first_join_date ~ 첫번째 homepage event 발생 시점까지의 시간을 첫 행에 입력
        else:
            date_range = pd.date_range(start=start_date, end=log.first_join_date.iloc[0], freq='M')
            extracted_months = set(date.strftime('%Y-%m') for date in date_range)
            log.at[first_index, 'diff'] = (log.at[first_index, 'event_time'] - pd.Timestamp(log.at[first_index, 'first_join_date'], tz='UTC')).total_seconds()
            bounce_month -= extracted_months
        
        # bounce_month 집합에 마지막 접속일 부터 10일 지난 시점의 달 추가(합집합)
        bounce_month = bounce_month.union(set([date.strftime('%Y-%m') for date in log.loc[(log_index - 1), 'event_time'] + timedelta(days= days_cri)]))

        # 이탈(1) 입력
        for month in bounce_month:
            tem[month] = 1

        # customer_id와 key에 대해 이탈 여부 매칭하는 딕셔너리 만들기
        res[customer] = [int(tem[key]) for key in sorted(tem.keys())]

    # 딕셔너리를 데이터프레임으로 변환
    res_df = pd.DataFrame.from_dict(res, orient='index', columns=column_names)
    
    return res_df


In [287]:
bounce_10 = bounce_judge(tem2, 10)

In [ ]:
bounce_11 = bounce_judge(tem2, 11)

In [ ]:
bounce_14 = bounce_judge(tem2, 14)

In [ ]:
bounce_1_5 = bounce_judge(tem2, 1.5)

In [278]:
len(bounce_10)

50685

In [279]:
tem2.customer_id.nunique()

50704

In [283]:
result = list(set(tem2.customer_id.unique()).difference(bounce_10.index))
len(result)

19

In [284]:
result

[6923.0,
 95249.0,
 8338.0,
 15640.0,
 98726.0,
 24236.0,
 47795.0,
 5813.0,
 90942.0,
 17857.0,
 86874.0,
 7008.0,
 92643.0,
 44527.0,
 18033.0,
 8178.0,
 39793.0,
 93557.0,
 78710.0]

In [282]:
tem2[tem2['customer_id'] == result[18]]

,session_id,event_time,event_name,customer_id,first_join_date
12204248,725a2118-dcec-4322-8c2b-cbff428acbd6,2022-07-01 21:47:45.792697+00:00,HOMEPAGE,78710.0,2022-06-28
12204249,725a2118-dcec-4322-8c2b-cbff428acbd6,2022-07-01 21:49:43.792697+00:00,ADD_TO_CART,78710.0,2022-06-28
12204250,725a2118-dcec-4322-8c2b-cbff428acbd6,2022-07-01 21:51:16.792697+00:00,ADD_TO_CART,78710.0,2022-06-28
12204251,725a2118-dcec-4322-8c2b-cbff428acbd6,2022-07-01 22:16:53.049205+00:00,BOOKING,78710.0,2022-06-28
12204252,725a2118-dcec-4322-8c2b-cbff428acbd6,2022-07-01 21:49:56.792697+00:00,ITEM_DETAIL,78710.0,2022-06-28
12204253,725a2118-dcec-4322-8c2b-cbff428acbd6,2022-07-01 21:51:50.792697+00:00,ITEM_DETAIL,78710.0,2022-06-28
12204254,725a2118-dcec-4322-8c2b-cbff428acbd6,2022-07-01 21:53:38.792697+00:00,SEARCH,78710.0,2022-06-28
12204255,725a2118-dcec-4322-8c2b-cbff428acbd6,2022-07-01 21:55:18.792697+00:00,CLICK,78710.0,2022-06-28
12204256,725a2118-dcec-4322-8c2b-cbff428acbd6,2022-07-01 21:57:05.792697+00:00,ITEM_DETAIL,78710.0,2022-06-28
12204257,725a2118-dcec-4322-8c2b-cbff428acbd6,2022-07-01 21:58:52.792697+00:00,CLICK,78710.0,2022-06-28
